In [61]:
import os
import sys
import glob
import json
from datetime import datetime
import datetime
import configparser
import pandas as pd
import numpy as np

In [62]:
import inspect
import sys
mlist = list(filter(lambda x: inspect.ismodule(x[1]), locals().items()))
vi = sys.version_info
print("version {0}.{1}.{2} of Python".format(vi.major, vi.minor, vi.micro))
for name, mod in mlist:
    if name.startswith("__"):
        continue
    if hasattr(mod, "__version__"):
        print("version {1} of {0}".format(name, mod.__version__))
del mod
del name

version 3.8.5 of Python
version 2.0.9 of json
version 1.1.3 of pd
version 1.19.2 of np


If you get the error message
  IOPub data rate exceeded.
 Try restarting the notebook as follows
 
>> jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [63]:
# load data using Python JSON module
dpath = os.path.join(".","takeout-20201113T114208Z-001","Takeout","Chrome","BrowserHistory.json")

with open(dpath,mode='r',  encoding="utf8") as f:
    data = json.loads(f.read())


In [64]:
with open(dpath, mode='r', encoding="utf8") as fp:
    data = json.load(fp)

keys = list(data.keys())
records = data[keys[0]]

rec_df = pd.DataFrame(records)
rec_df["ndt"] = rec_df["time_usec"].apply(lambda x: datetime.datetime.fromtimestamp(int(x/10**6)))
rec_df["pdt"] = pd.to_datetime(rec_df["ndt"])
rec_df["month"] = rec_df["pdt"].apply(lambda x:x.month)
rec_df["month"] = rec_df["pdt"].apply(lambda x:x.month)
rec_df["day"] = rec_df["pdt"].apply(lambda x:x.day)
rec_df["hour"] = rec_df["pdt"].apply(lambda x:x.hour)
rec_df.head()

,favicon_url,page_transition,title,url,client_id,time_usec,ndt,pdt,month,day,hour
0,https://www.google.com/favicon.ico,LINK,Google Takeout,https://takeout.google.com/?pli=1,bWlnfwc3heyNRsaQWmnPVw==,1605267699751888,2020-11-13 06:41:39,2020-11-13 06:41:39,11,13,6
1,https://www.google.com/favicon.ico,LINK,Google Takeout,https://takeout.google.com/?pli=1,bWlnfwc3heyNRsaQWmnPVw==,1605267694964460,2020-11-13 06:41:34,2020-11-13 06:41:34,11,13,6
2,https://www.google.com/favicon.ico,LINK,Google Takeout,https://takeout.google.com/?pli=1,bWlnfwc3heyNRsaQWmnPVw==,1605267678976501,2020-11-13 06:41:18,2020-11-13 06:41:18,11,13,6
3,https://www.google.com/favicon.ico,LINK,Google Takeout,https://takeout.google.com/?pli=1,bWlnfwc3heyNRsaQWmnPVw==,1605267653440065,2020-11-13 06:40:53,2020-11-13 06:40:53,11,13,6
4,https://support.google.com/favicon.ico,LINK,How to download your Google data - Google Acco...,https://support.google.com/accounts/answer/302...,bWlnfwc3heyNRsaQWmnPVw==,1605267634155673,2020-11-13 06:40:34,2020-11-13 06:40:34,11,13,6


In [65]:
oct = rec_df[rec_df["month"]==10]
oct.tail()
gcols = ["title", "url", "month", "day"]#, "day"]
cntcol = "hour"
lcols = gcols + [cntcol]
gdf = rec_df[lcols].groupby(by=gcols).agg(["count"])
df = gdf.droplevel(axis=1, level=1).rename(columns={cntcol:"cnt"}).sort_values(by="cnt", ascending=False)
df.reset_index(inplace=True)
df.head()

,title,url,month,day,cnt
0,Bloomberg.com,https://www.bloomberg.com/,11,10,207
1,Bloomberg.com,https://www.bloomberg.com/,11,9,159
2,Bloomberg.com,https://www.bloomberg.com/,11,11,131
3,Bloomberg.com,https://www.bloomberg.com/,11,4,114
4,Enter The Data,http://localhost:5000/,9,28,107


In [96]:
z = df.loc[df["title"].str.upper().str.contains("COVID")]
z.reset_index(drop=True, inplace=True)
print(z.shape)
z.loc[30:60]

(135, 5)


,title,url,month,day,cnt
30,Donald Trump continues bizarre appeals to subu...,https://www.cnn.com/2020/10/18/politics/donald...,10,18,1
31,COVID-19 (Coronavirus Disease) | Wisconsin Dep...,https://www.dhs.wisconsin.gov/covid-19/index.htm,10,20,1
32,Tracking Covid-19 cases in the US,https://www.cnn.com/interactive/2020/health/co...,10,22,1
33,﻿﻿﻿No Laughing Matter: the Data Strongly Suppo...,https://medium.com/datadriveninvestor/no-laugh...,10,22,1
34,No Laughing Matter: the Data Strongly Supports...,https://medium.com/datadriveninvestor/no-laugh...,10,22,1
35,No Laughing Matter: the Data Strongly Supports...,https://medium.com/datadriveninvestor/no-laugh...,10,23,1
36,Epidemiologists Stray From the Covid Herd - WSJ,https://www.wsj.com/articles/epidemiologists-s...,10,24,1
37,Labour markets during the Covid-19 crisis: A p...,https://voxeu.org/article/labour-markets-durin...,10,27,4
38,﻿﻿The Covid 19 health issue — ANYTHING SHE SAYS,https://concillier.squarespace.com/datasets/co...,10,27,1
39,leading indicators labor market us covid - Goo...,https://www.google.com/search?biw=1673&bih=709...,10,27,1


In [79]:
if False:
    df.sort_values(by=["month", "day"], inplace=True)
    x = df.copy()
    x = x.loc[x["month"] >= 11]
    #x = x.loc[np.logical_and(x["day"] >= 27, x["day"] <= 29)]
    x = x.loc[x["cnt"]>1]
    x = x
    print(x.shape)
    #print(x.groupby(by="day").agg(["count"]))
    x.reset_index(drop=True, inplace=True)

    heading_properties = [('font-size', '30px')]
    cell_properties = [('font-size', '22px')]
    dfstyle = [dict(selector="th", props=heading_properties),\
     dict(selector="td", props=cell_properties)]
    x.style.set_table_styles(dfstyle)
    start = 302
    x[start:start+30]


## November

In [87]:
df.sort_values(by=["month", "day"], inplace=True)
x = df.copy()
x = x.loc[x["month"] >= 11]
#x = x.loc[np.logical_and(x["day"] >= 27, x["day"] <= 29)]
x = x.loc[x["cnt"]>1]
x = x
print(x.shape)
#print(x.groupby(by="day").agg(["count"]))
x.reset_index(drop=True, inplace=True)
heading_properties = [('font-size', '30px')]
cell_properties = [('font-size', '22px')]
dfstyle = [dict(selector="th", props=heading_properties),\
 dict(selector="td", props=cell_properties)]
x.style.set_table_styles(dfstyle)
start = 278
x[start:start+35]


(828, 5)


,title,url,month,day,cnt
278,Glacier Bay Touchless LED Single-Handle Pull-D...,https://www.homedepot.com/p/Glacier-Bay-Touchl...,11,4,6
279,FiveThirtyEight | Nate Silver’s FiveThirtyEigh...,https://fivethirtyeight.com/,11,4,6
280,Untitled spreadsheet - Google Sheets,https://docs.google.com/spreadsheets/d/1IDg6nu...,11,4,6
281,(3) Home / Twitter,https://twitter.com/home,11,4,5
282,umich_cs - Jupyter Notebook,http://localhost:8888/notebooks/_notebooks/umi...,11,4,5
283,Kitchen faucet at Lowes.com: Search Results,https://www.lowes.com/search?searchTerm=kitche...,11,4,4
284,"Google Calendar - Week of November 2, 2020",https://calendar.google.com/calendar/u/0/r/week,11,4,4
285,"The Wall Street Journal - Breaking News, Busin...",https://www.wsj.com/,11,4,4
286,"The Wall Street Journal - Breaking News, Busin...",https://www.wsj.com/?mod=nav_top_section,11,4,3
287,2020 US Presidential Election Results: Live Ma...,https://abcnews.go.com/Elections/2020-us-presi...,11,4,3


In [78]:
if False:
    oct2_df = df.loc[np.logical_and(df["day"] > 15, df["month"] == 10)]
    nov_df = df.loc[df["month"] == 11]
    oct2_df.reset_index(inplace=True)
    nov_df.reset_index(inplace=True)
    print(oct2_df.shape, nov_df.shape)

In [77]:
if False:
    #print(oct2_df.shape)
    x = oct2_df.loc[np.logical_and(oct2_df["cnt"] <= 10, oct2_df["cnt"]>= 2)].copy()
    x.sort_values(by="cnt", inplace=True)
    x.reset_index(inplace=True, drop = True)
    print("Oct 2nd half {0}".format(x.shape[0]))

    n=4
    incr = 20
    s = 0
    for i in range(n):
        s = s+incr
    e = s + incr 
    x.loc[s:e]